## _*The EOH (Evolution of Hamiltonian) Algorithm*_

This notebook demonstrates how to use the `Qiskit Aqua` library to invoke the EOH algorithm and process the result.

Further information may be found for the algorithms in the online [Aqua documentation](https://qiskit.org/documentation/aqua/algorithms.html).

For this particular demonstration, we illustrate the `EOH` algorithm. First, two `Operator` instances we created are randomly generated Hamiltonians.

In [1]:
import numpy as np
from qiskit_aqua.operator import Operator

num_qubits = 2
temp = np.random.random((2 ** num_qubits, 2 ** num_qubits))
qubitOp = Operator(matrix=temp + temp.T)
temp = np.random.random((2 ** num_qubits, 2 ** num_qubits))
evoOp = Operator(matrix=temp + temp.T)

For EOH, we would like to evolve some initial state (e.g. the uniform superposition state) with `evoOp` and do a measurement using `qubitOp`. Below, we illustrate how such an example dynamics process can be easily prepared.

In [2]:
from qiskit_aqua.input import get_input_instance

params = {
    'problem': {
        'name': 'eoh'
    },
    'algorithm': {
        'name': 'EOH',
        'num_time_slices': 1
    },
    'initial_state': {
        'name': 'CUSTOM',
        'state': 'uniform'
    },
    'backend': {
        'name': 'statevector_simulator'
    }
}
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp
algo_input.add_aux_op(evoOp)

With all the necessary pieces prepared, we can then proceed to run the algorithm and examine the result.

In [3]:
from qiskit_aqua import run_algorithm

ret = run_algorithm(params, algo_input)
print('The result is\n{}'.format(ret))

The result is
{'avg': (3.1861092137132543-1.184340832586723e-16j), 'std_dev': 0.0}
